In [1]:
import sys

import numpy as np
import math
from carSimulator import CarSimulator
from numpy.linalg import inv

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def getControlJacobian(sim, u, x):
    L = sim.L
#     B = np.zeros((3,2))
    B = np.array([[np.cos(x[2]), 0],
                  [np.sin(x[2]), 0],
                  [np.tan(u[1]) / L, 1/np.cos(u[1])**2 * u[0] / L]],
                 dtype=np.float32)
    
    return B

In [5]:
def main():

    sim = CarSimulator()
    u = [.1, .2]  # fixed control signal
    
    # Kalman estimates
    s = np.zeros(3)
    S = np.zeros((3,3))
    
    # initial at true state
    s[0] = sim.x
    s[1] = sim.y
    s[2] = sim.theta
    # with noisy constant
    np.fill_diagonal(S, 0.1)
    
    W = np.identity(4) * sim.observationNoise
    Q = np.identity(3) * sim.dynamicsNoise
    
    A = np.identity(3)
    for t in range(10000):
        sim.step(u)
        y_meassured = sim.getRealNoisyObservation()
        
        # get the linear observation model
        C = sim.getObservationJacobianAtState(s)
        y_mean = sim.getMeanObservationAtState(s)
        
        # get the linear control model
        B = getControlJacobian(sim, u, s)
        a = np.dot(B, u)
        
        # Kalman filter
        # 1) predict, s^ for mean, S^ for covariance matrix of the estimated state
        shat = A@s + a
        Shat = Q + A@S@A.T
        # 2) update, first calculate the Kalman Gain
        K = Shat@C.T@inv(W + C@Shat@C.T)
        # then calculate posterior s and S for updated mean and convariance
        c = y_mean - C@s
        S = Shat - K@C@Shat
        s = shat + K@(y_meassured - C@shat - c)

        # code to display a covariance
        sim.gaussianMeansToDraw = [s[:2]]
        sim.gaussianCovsToDraw =[S[:2, :2]]

        
        # tracking error
        error = np.abs(s - np.array([sim.x, sim.y, sim.theta]))
        print("estim error", np.max(error))
        
        


In [7]:
main()

estim error 0.1527171160068334
estim error 0.13303819593567973
estim error 0.20861435069946352
estim error 0.31195270733986635
estim error 0.24615380381907268
estim error 0.27858686410335926
estim error 0.1506755483258099
estim error 0.15652897109801459
estim error 0.10952478305946975
estim error 0.0733846360298954
estim error 0.1854006854197846
estim error 0.09120448742679015
estim error 0.1425468115000863
estim error 0.16620694242393164
estim error 0.09557370917254082
estim error 0.06738191873500043
estim error 0.14130784941527572
estim error 0.08985598760000423
estim error 0.06907009116313567
estim error 0.17081145307462023
estim error 0.19492889668994207
estim error 0.23104431389710653
estim error 0.19635506732276445
estim error 0.2280738153212104
estim error 0.1840139574987602
estim error 0.2315062072278855
estim error 0.15999256398510653
estim error 0.07216967194118634
estim error 0.1985280367447227
estim error 0.07042559901615464
estim error 0.022186603323109982
estim error 0.12

error: video system not initialized